In [ ]:
import pandas as pd
#xs_dict={'ZH_HToCC_ZToLL_M125_13TeV_powheg_pythia8':0.002342},
lumi= 41500 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib.offsetbox import AnchoredText
from coffea.util import load
from coffea.hist import plot
from coffea import hist
np.seterr(all="ignore")
import os,math,re,json,shutil

plt.style.use(hep.style.ROOT)


data_err_opts = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color': 'k',
    'elinewidth': 1,
}
data_err_opts2 = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color': 'b',
    'elinewidth': 1,
}
from cycler import cycler
import matplotlib as mpl
# colors=["#F44336","#E91E63","#9C27B0","#673AB7","#3F51B5","#2196F3","#03A9F4","#00BCD4","#009688","#4CAF50","#8BC34A","#CDDC39","#FFEB3B","#FFC107","#FF9800","#FF5722","#795548","#BDBDBD","#9E9E9E","#616161","#90BED4","#607D8B","#455A64"]

#colors = ["#666666","#1D6996","#38A6A5","#0F8554","#73AF48","#EDAD08","#E17C05","#CC503E","#554e99","#6f4e99","#854e99","#994e85","#666666"]
colors = ["darkblue","lightpink","lightcoral","aqua","greenyellow","olivedrab","palegreen","fuchsia","red","darkviolet","firebrick","plum","orange"]
mpl.rcParams["axes.prop_cycle"] = cycler('color', colors)
from matplotlib.offsetbox import AnchoredText

In [ ]:
from utils.xs_scaler import scale_xs

with open('metadata/mergemap_2017_Zll_used_keys.json') as json_file:  
    merge_map = json.load(json_file)
#with open('metadata/plotmap.json') as pltf:
#    var_map = json.load(pltf)

In [ ]:
data = load('datasamples_2017_Zll_used.coffea')
higgs = load('mcsamples_2017_higgs_Zll_used.coffea')
vjets = load('mcsamples_2017_vjets_Zll_used.coffea')
vjets_ext = load('mcsamples_2017_vjets_ext_Zll_used.coffea')
other = load('mcsamples_2017_other_Zll_used.coffea')

In [ ]:
data

In [ ]:
higgs

In [ ]:
other

In [ ]:
vjets

In [ ]:
vjets_ext

In [ ]:
data['sumw']

In [ ]:
higgs['sumw']

In [ ]:
other['sumw']

In [ ]:
vjets['sumw']

In [ ]:
vjets_ext['sumw']

In [ ]:
data = load('datasamples_2017_Zll_used.coffea')
higgs = load('mcsamples_2017_higgs_Zll_used.coffea')
vjets = load('mcsamples_2017_vjets_Zll_used.coffea')
#vjets_ext = load('mcsamples_2017_vjets_ext_Zll_used.coffea')
other = load('mcsamples_2017_other_Zll_used.coffea')

#ch = ['mumu','ee']
ch = ['mumu']

#regions = ['SR','SR_2LL','SR_2LH','CR_Zcc_2LL','CR_Zcc_2LH','CR_Z_LF_2LL','CR_Z_LF_2LH','CR_Z_HF_2LL','CR_Z_HF_2LH','CR_t_tbar_2LL','CR_t_tbar_2LH']
#regions = ['SR_2LL','SR_2LH']
regions = ['SR_2LL']
#region_map = {'SR_2LL':'Low $p_T(V)$ SR','SR_2LH':'High $p_T(V)$ SR'}
region_map = {'SR_2LL':'Low $p_T(V)$ SR'}

#for var in ['leading_jetflav_btagDeepFlavCvL','subleading_jetflav_btagDeepFlavCvL','leading_jetflav_btagDeepFlavCvB','leading_jetflav_btagDeepFlavB','leading_jetflav_btagDeepFlavC']:
for var in ['leading_jetflav_btagDeepFlavCvL']:
    scales=1
    #5e4
#     if var in [ 'nj','nele','nmu','njmet']:scales=1000
    #events = something['sumw']
    #print(a_test_output[var],41500*scales,events)
    #hmpf = scale_xs()
    higgs[var]=scale_xs(higgs[var],415.00*scales,higgs['sumw'])
    other[var]=scale_xs(other[var],415.00,other['sumw'])
    vjets[var]=scale_xs(vjets[var],415.00,vjets['sumw'])
    #vjets[var]=scale_xs(vjets[var],41500,vjets['sumw']+vjets_ext['sumw'])
    #vjets_ext[var]=scale_xs(vjets_ext[var],41500,vjets['sumw']+vjets_ext['sumw'])
    
    
    data[var]=data[var].group("datasetSplit",hist.Cat("plotgroup", "plotgroup"), {"Data" : merge_map['Data']})
    higgs[var]=higgs[var].group("dataset",hist.Cat("plotgroup", "plotgroup"),merge_map)
    #other[var]=other[var].group("datasetSplit",hist.Cat("plotgroup", "plotgroup"), merge_map['Single top','t#bar{t}','VV+other','VV+cc'])
    #vjets[var]=vjets[var].group("datasetSplit",hist.Cat("plotgroup", "plotgroup"),merge_map['Z+l','Z+b','Z+c'])
    #vjets_ext[var]=vjets_ext[var].group("datasetSplit",hist.Cat("plotgroup", "plotgroup"),merge_map['Z+l','Z+b','Z+c'])
    other[var]=other[var].group("datasetSplit",hist.Cat("plotgroup", "plotgroup"), merge_map)
    vjets[var]=vjets[var].group("datasetSplit",hist.Cat("plotgroup", "plotgroup"), merge_map)
    #vjets_ext[var]=vjets_ext[var].group("datasetSplit",hist.Cat("plotgroup", "plotgroup"), merge_map)
    
    
    if var == 'nbj' or var == 'ncj' or 'jetpt_' in var or 'jetcsv_' in var  or 'jetpn_' in var or 'dr' in var or var=='MET_covXY' : continue    
    
    if var == 'cutflow':break

    for chs in ch:
        for region in regions:
                print(var,chs,region)
                if 'SR' not in region and '_dphi' in var: continue
                fig, ((ax),(rax)) = plt.subplots(2, 1, figsize=(12, 12), gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
                hep.cms.label("Work in progress", data=True, lumi=41.5, year=2017,ax=ax)

                fig.subplots_adjust(hspace=.07)
#                     ax = plot.plot1d(outputWWl_b[var].sum("flav").integrate("lepflav",chs).integrate("region",region),overlay="plotgroup",stack=True,clear=False,order=['Z+jets','W+jets','tt-dilep','tt-semilep','ST','WW','WZ','ZZ'],ax=ax)
#                     ax = plot.plot1d(outputWWl_b[var].sum("flav").integrate("lepflav",chs).integrate("region",['DY_CRl','DY_CRb','DY_CRc']),overlay="plotgroup",stack=True,clear=False,order=['Z+jets','W+jets','tt-dilep','tt-semilep','ST','WW','WZ','ZZ'],ax=ax)
#                 hmc=outputWWl_b[var].sum("flav").integrate("lepflav",chs).integrate("region",'DY_CRl').add(outputWWl_b[var].sum("flav").integrate("lepflav",chs).integrate("region",'DY_CRb'))
#                 hmc.add(outputWWl_b[var].sum("flav").integrate("lepflav",chs).integrate("region",'DY_CRc'))
                
                #hmc = outputWWl_b[var].integrate("lepflav",chs).integrate("region",region)
                #ax = plot.plot1d(hmc.sum('flav'),overlay="plotgroup",stack=True,clear=False,order=['Z+jets','W+jets','tt-dilep','tt-semilep','ST','WW','WZ','ZZ'],ax=ax)
                #plot.plot1d(hmc.integrate('plotgroup','Z+jets'),overlay="flav",stack=True,ax=ax,clear=False)
                #hdata = outputWWl_data[var].integrate("lepflav",chs).integrate('region',region).integrate('plotgroup','data_%s'%(chs)).sum('flav')
                
                plot.plot1d(other[var].sum("flav").sum("dataset").integrate("lepflav",chs).integrate('region',region),stack=True,
                            overlay='plotgroup',clear=False,ax=ax)
                plot.plot1d(vjets[var].sum("flav").sum("dataset").integrate("lepflav",chs).integrate('region',region),stack=True,
                            overlay='plotgroup',clear=False,ax=ax)
                plot.plot1d(higgs[var].sum("flav").sum("datasetSplit").integrate("lepflav",chs).integrate('region',region),stack=False,
                            overlay='plotgroup',clear=False,ax=ax)
                #plot.plot1d(vjets_ext[var].sum("flav").sum("dataset").integrate("lepflav",chs).integrate('region',region),stack=True,
                #            overlay='plotgroup',clear=False,ax=ax)
                #print(something[var].sum("flav").sum("dataset").sum("datasetSplit").integrate("lepflav",chs).integrate('region',region))
                plot.plot1d(data[var].sum("flav").sum("dataset").sum("plotgroup").integrate("lepflav",chs).integrate('region',region),clear=False,error_opts=data_err_opts,ax=ax)
                #plot.plot1d(data_output[var].sum("flav").sum("dataset").integrate("lepflav",chs).integrate('region',region),clear=False,ax=ax)
                #plot.plot1d(hdata,clear=False,error_opts=data_err_opts,ax=ax)
                
#                 print(sum(hdata.values()[()]),sum(hmc.sum('plotgroup').values()[()]))
                #print(sum(outputWWl_s[var].sum("flav").integrate("lepflav",chs).integrate('region',region).integrate('dataset','gchcWW2L2Nu').values()[()]),sum(hdata.values()[()]))
                
                #rax = plot.plotratio(num=hdata,
                #                                denom=hmc.sum('plotgroup').sum('flav'),
                #                                        ax=rax,
                #                                        error_opts=data_err_opts,
                #                                        denom_fill_opts={},                                              
        #                                       
                #                                       unc='num',
                #                                   clear=False
                #                                       )
                  
#                 
                rax.set_ylim(0.5,1.5)
                rax.set_ylabel('Data/MC')
                #rax.set_xlabel(var_map[var])
                rax.set_xlabel(var)
                ax.set_xlabel('')
                chl = chs
                if chs =='mumu' :chl='$\mu\mu$'
                #if chs =='emu' :chs='e$\mu$'
                at = AnchoredText(chl+"  "+region_map[region]+"\n" +r"H$\rightarrow c\overline{c}, 2\ell$"                                                         
                                            , loc='upper left',frameon=False)
                ax.add_artist(at)
                leg_label = ax.get_legend_handles_labels()[1][0:]
                for (i,lab) in enumerate(leg_label):
                    if '#' in lab:
                        leg_label[i] = lab.split('#')[0] + r"$\bar{" +lab.split('#')[1].split('{')[1][0] + (r"})$" if 't' not in lab else r"}$" )
                leg_label[-1]='Data'
                # leg_label[-2]='Signalx%d' %(scales)
#                 print(ax.get_legend_handles_labels()[1])
                ax.legend(loc="upper right",handles=ax.get_legend_handles_labels()[0][0:],ncol=2,labels=leg_label,fontsize=18)
                hep.mpl_magic(ax= ax)            

In [ ]:

data = load('datasamples_2017_Zll_used.coffea')
higgs = load('mcsamples_2017_higgs_Zll_used.coffea')
vjets = load('mcsamples_2017_vjets_Zll_used.coffea')
#vjets_ext = load('mcsamples_2017_vjets_ext_Zll_used.coffea')
other = load('mcsamples_2017_other_Zll_used.coffea')

#ch = ['mumu','ee']
ch = ['mumu']

#regions = ['SR','SR_2LL','SR_2LH','CR_Zcc_2LL','CR_Zcc_2LH','CR_Z_LF_2LL','CR_Z_LF_2LH','CR_Z_HF_2LL','CR_Z_HF_2LH','CR_t_tbar_2LL','CR_t_tbar_2LH']
#regions = ['SR_2LL','SR_2LH']
regions = ['SR_2LL']
#region_map = {'SR_2LL':'Low $p_T(V)$ SR','SR_2LH':'High $p_T(V)$ SR'}
region_map = {'SR_2LL':'Low $p_T(V)$ SR'}

#for var in ['leading_jetflav_btagDeepFlavCvL','subleading_jetflav_btagDeepFlavCvL','leading_jetflav_btagDeepFlavCvB','leading_jetflav_btagDeepFlavB','leading_jetflav_btagDeepFlavC']:
for var in ['leading_jetflav_btagDeepFlavCvL']:
    scales=1
    
    data[var]=data[var].group("datasetSplit",hist.Cat("plotgroup", "plotgroup"), {"Data" : merge_map['Data']})
    higgs[var]=higgs[var].group("dataset",hist.Cat("plotgroup", "plotgroup"),merge_map)
    other[var]=other[var].group("datasetSplit",hist.Cat("plotgroup", "plotgroup"), merge_map)
    vjets[var]=vjets[var].group("datasetSplit",hist.Cat("plotgroup", "plotgroup"), merge_map)
    #vjets_ext[var]=vjets_ext[var].group("datasetSplit",hist.Cat("plotgroup", "plotgroup"), merge_map)
    
    
    if var == 'nbj' or var == 'ncj' or 'jetpt_' in var or 'jetcsv_' in var  or 'jetpn_' in var or 'dr' in var or var=='MET_covXY' : continue    
    
    if var == 'cutflow':break

    for chs in ch:
        for region in regions:
                print(var,chs,region)
                if 'SR' not in region and '_dphi' in var: continue
                fig, ((ax),(rax)) = plt.subplots(2, 1, figsize=(12, 12), gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
                hep.cms.label("Work in progress", data=True, lumi=41.5, year=2017,ax=ax)

                fig.subplots_adjust(hspace=.07)  
                
                plot.plot1d(other[var].sum("flav").sum("dataset").integrate("lepflav",chs).integrate('region',region),stack=True,
                            overlay='plotgroup',clear=False,ax=ax)
                plot.plot1d(vjets[var].sum("flav").sum("dataset").integrate("lepflav",chs).integrate('region',region),stack=True,
                            overlay='plotgroup',clear=False,ax=ax)
                plot.plot1d(higgs[var].sum("flav").sum("datasetSplit").integrate("lepflav",chs).integrate('region',region),stack=False,
                            overlay='plotgroup',clear=False,ax=ax)
                #plot.plot1d(vjets_ext[var].sum("flav").sum("dataset").integrate("lepflav",chs).integrate('region',region),stack=True,
                #            overlay='plotgroup',clear=False,ax=ax)
                #print(something[var].sum("flav").sum("dataset").sum("datasetSplit").integrate("lepflav",chs).integrate('region',region))
                plot.plot1d(data[var].sum("flav").sum("dataset").sum("plotgroup").integrate("lepflav",chs).integrate('region',region),clear=False,error_opts=data_err_opts,ax=ax)
                
                #rax = plot.plotratio(num=hdata,
                #                                denom=hmc.sum('plotgroup').sum('flav'),
                #                                        ax=rax,
                #                                        error_opts=data_err_opts,
                #                                        denom_fill_opts={},                                              
        #                                       
                #                                       unc='num',
                #                                   clear=False
                #                                       )
                  
#                 
                rax.set_ylim(0.5,1.5)
                rax.set_ylabel('Data/MC')
                rax.set_xlabel(var)
                ax.set_xlabel('')
                chl = chs
                if chs =='mumu' :chl='$\mu\mu$'
                #if chs =='emu' :chs='e$\mu$'
                at = AnchoredText(chl+"  "+region_map[region]+"\n" +r"H$\rightarrow c\overline{c}, 2\ell$"                                                         
                                            , loc='upper left',frameon=False)
                ax.add_artist(at)
                leg_label = ax.get_legend_handles_labels()[1][0:]
                for (i,lab) in enumerate(leg_label):
                    if '#' in lab:
                        leg_label[i] = lab.split('#')[0] + r"$\bar{" +lab.split('#')[1].split('{')[1][0] + (r"})$" if 't' not in lab else r"}$" )
                leg_label[-1]='Data'
                # leg_label[-2]='Signalx%d' %(scales)
#                 print(ax.get_legend_handles_labels()[1])
                ax.legend(loc="upper right",handles=ax.get_legend_handles_labels()[0][0:],ncol=2,labels=leg_label,fontsize=18)
                hep.mpl_magic(ax= ax)